In [11]:
import dlib
import cv2
import numpy as np


In [12]:
model_path="/home/jal/.deepface/weights/vgg_face_weights.h5"
# Load the pre-trained face detection model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../raw_data/shape_predictor_81_face_landmarks.dat')


In [14]:
import os

In [15]:
os.getcwd()


'/home/jal/code/stan-kondratev/celebrities-identification-quest/notebooks'

In [17]:
folder_path= "../raw_data/output_imdb_top100/"

file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
file_list

['../raw_data/output_imdb_top100/ellen_burstyn.jpg',
 '../raw_data/output_imdb_top100/julia_roberts.jpg',
 '../raw_data/output_imdb_top100/meryl_streep.jpg',
 '../raw_data/output_imdb_top100/jack_lemmon.jpg',
 '../raw_data/output_imdb_top100/jennifer_lawrence.jpg',
 '../raw_data/output_imdb_top100/kate_mara.jpg',
 '../raw_data/output_imdb_top100/adele_exarchopoulos.jpg',
 '../raw_data/output_imdb_top100/marilyn_monroe.jpg',
 '../raw_data/output_imdb_top100/michael_caine.jpg',
 '../raw_data/output_imdb_top100/spencer_tracy.jpg',
 '../raw_data/output_imdb_top100/mel_gibson.jpg',
 '../raw_data/output_imdb_top100/amanda_plummer.jpg',
 '../raw_data/output_imdb_top100/johnny_depp.jpg',
 '../raw_data/output_imdb_top100/brad_pitt.jpg',
 '../raw_data/output_imdb_top100/demi_moore.jpg',
 '../raw_data/output_imdb_top100/jessica_chastain.jpg',
 '../raw_data/output_imdb_top100/charlize_theron.jpg',
 '../raw_data/output_imdb_top100/george_c._scott.jpg',
 '../raw_data/output_imdb_top100/gene_hackman.

In [18]:
images = [f.replace('.jpg', '') for f in file_list]
for image_path in images:
    image = cv2.imread(image_path+'.jpg')
    # Convert the image to grayscale for face detection
    gray = cv2.cvtColor(image, cv2.GRCOLOR_BGR2GRAY)
    # Detect faces in the image
    faces = detector(gray)
    # We take only the first detected face, if exist
    if len(faces) == 0:
        continue
    face = faces[0]
    
    # We extract the landmarks   
    landmarks = predictor(image, face)
    landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

    # Calculate the center of the eyes
    left_eye = np.mean(landmarks[36:42], axis=0)
    right_eye = np.mean(landmarks[42:48], axis=0)
    nose = landmarks[30]

    # Calculate the angle between the eyes and the horizontal line
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dY, dX))

    # Rotate the image around the center of the face rectangle
    (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())
    center = (x + w // 2, y + h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    aligned_face = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_CUBIC)
    if ((aligned_face.shape[0] == 0) or (aligned_face.shape[1] == 0) or (aligned_face.shape[2] == 0)):
        continue
    
    # Re apply the face crop on the aligned image
    gray = cv2.cvtColor(aligned_face, cv2.COLOR_BGR2GRAY)
    faces_aligned = detector(gray)
    if not faces_aligned:
        continue
    x, y, w, h = faces_aligned[0].left(), faces_aligned[0].top(), faces_aligned[0].width(), faces_aligned[0].height()
    processed_face = aligned_face[y:y+h, x:x+w]
    if ((processed_face.shape[0] == 0) or (processed_face.shape[1] == 0) or (processed_face.shape[2] == 0)):
        continue
    
    # Save the final preprocessed image
    save_path = image_path+"_face_aligned.jpg"
    
    # black and white output:
    cv2.imwrite(save_path, processed_face) #cv2.cvtColor(processed_face, cv2.COLOR_BGR2GRAY)
    #cv2.imwrite(save_path, face_image)
    print(f"Face image saved: {save_path}")

Face image saved: ../raw_data/output_imdb_top100/ellen_burstyn_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top100/julia_roberts_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top100/meryl_streep_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top100/jack_lemmon_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top100/jennifer_lawrence_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top100/kate_mara_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top100/adele_exarchopoulos_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top100/michael_caine_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top100/spencer_tracy_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top100/mel_gibson_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top100/johnny_depp_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top100/brad_pitt_face_aligned.jpg
Face image saved: ../raw_data/output_imdb_top1

In [12]:
os.getcwd()

'/home/jal/code/stan-kondratev/celebrities-identification-quest/notebooks'

In [6]:
import os as os
folder_path = "../raw_data/output_imdb_top100"

file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]



In [7]:
file_list

['../raw_data/output_imdb_top100/ellen_burstyn.jpg',
 '../raw_data/output_imdb_top100/julia_roberts.jpg',
 '../raw_data/output_imdb_top100/meryl_streep.jpg',
 '../raw_data/output_imdb_top100/jack_lemmon.jpg',
 '../raw_data/output_imdb_top100/jennifer_lawrence.jpg',
 '../raw_data/output_imdb_top100/kate_mara.jpg',
 '../raw_data/output_imdb_top100/adele_exarchopoulos.jpg',
 '../raw_data/output_imdb_top100/marilyn_monroe.jpg',
 '../raw_data/output_imdb_top100/michael_caine.jpg',
 '../raw_data/output_imdb_top100/spencer_tracy.jpg',
 '../raw_data/output_imdb_top100/mel_gibson.jpg',
 '../raw_data/output_imdb_top100/amanda_plummer.jpg',
 '../raw_data/output_imdb_top100/johnny_depp.jpg',
 '../raw_data/output_imdb_top100/brad_pitt.jpg',
 '../raw_data/output_imdb_top100/demi_moore.jpg',
 '../raw_data/output_imdb_top100/jessica_chastain.jpg',
 '../raw_data/output_imdb_top100/charlize_theron.jpg',
 '../raw_data/output_imdb_top100/george_c._scott.jpg',
 '../raw_data/output_imdb_top100/gene_hackman.

In [9]:
import os as os

folder_path = "../raw_data/output_imdb_top100"
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]


# Reduce maximum size of file to a specified width
for image in file_list:
    # Define max image width
    max_width = 600
    # Retrieve image from the file list unchanged
    src = cv2.imread(image,cv2.IMREAD_UNCHANGED)

    # check dimension of the retrieved image
    width = int(src.shape[1])
    height = int(src.shape[0])

    ratio = width/height
    reduce_factor = max_width/width

    if reduce_factor <= 1:
        dsize = (max_width, int(height*reduce_factor))
        output = cv2.resize(src,dsize,interpolation=cv2.INTER_LANCZOS4)
        cv2.imwrite(image,output) 